In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml import Pipeline
from preprocessing.preprocessor import split_data, class_imbalance
from kafka import KafkaProducer
import json
import time

%load_ext autoreload
%autoreload 2

In [2]:
spark = SparkSession.builder.appName("KafkaProducer").getOrCreate()

24/12/05 23:29:40 WARN Utils: Your hostname, Nikhils-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.206 instead (on interface en0)
24/12/05 23:29:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 23:29:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/05 23:29:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
spark.stop()

ConnectionRefusedError: [Errno 61] Connection refused

In [3]:
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))
topic = 'transactions'

In [4]:
data = spark.read.csv("./../dataset/creditcard.csv", header=True, inferSchema=True).cache()
data.show(5)

24/12/05 23:29:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

In [5]:
for i in range(1, 100):
    row = data.take(i)[0]
    producer.send(topic, value=row.asDict())
    time.sleep(2)
producer.flush()

KeyboardInterrupt: 